<a href="https://colab.research.google.com/github/veiro/tesis-imputacion-datos/blob/main/codigo/imputadores/Xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEER
1. Cambiar en enterno a GPU y la variable cuda= True
2. Para ejecutar todo el dataset, MODO_DESARROLLO=False, USAR_GOOGLE_DRIVE =  False y ejecutar un docker local de colab por que demora mas de 12 hrs.
3. tomado de https://zenodo.org/records/10016161

## 1. Requirements

In [ ]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
if (USAR_GOOGLE_DRIVE):
  from google.colab import drive
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_UTILS= "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/utils"
else:
  PATH_DATA_PROCESADA = "C:\\Users\\aveiro\\Desktop\\Tesis\\data\\data-procesada"
  PATH_UTILS = "C:\\Users\\aveiro\\Desktop\\Tesis\\Codigo\\utils\\"

sys.path.append(PATH_UTILS)

import utils as utils
import importlib
importlib.reload(utils)

if (30 ==  utils.version()):
  print("version correcta")
else:
  raise Exception("Version de Util vieja")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
version correcta


In [ ]:
!python3 -m pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.5
cufflinks==0.17.3
cupy-cuda12x==12.2.0

## 3. Prepare Data

## Armo juego de datos entramiento y test


In [ ]:
df_train_encoded, df_train_encoded_missing, mask_train_open = utils.obtenerDatosTrain(MODO_DESARROLLO, PATH_DATA_PROCESADA)
df_test_encoded, df_test_encoded_missing, mask_test_open = utils.obtenerDatosTest(MODO_DESARROLLO, PATH_DATA_PROCESADA)

df_test_encoded = pd.DataFrame(df_test_encoded, columns = df_train_encoded.columns)
df_test_encoded_missing = pd.DataFrame(df_test_encoded_missing, columns = df_train_encoded.columns)



In [ ]:
utils.mostrarDatos(df_train_encoded)

Nombre datos: 

--------------------------------------------------------------------------------

Index(['Country_Region_Australia', 'Country_Region_Belgium',
       'Country_Region_Brazil', 'Country_Region_Canada',
       'Country_Region_Chile', 'Country_Region_China',
       'Country_Region_Colombia', 'Country_Region_Denmark',
       'Country_Region_France', 'Country_Region_Germany',
       'Country_Region_India', 'Country_Region_Italy', 'Country_Region_Japan',
       'Country_Region_Malaysia', 'Country_Region_Mexico',
       'Country_Region_Netherlands', 'Country_Region_New Zealand',
       'Country_Region_Pakistan', 'Country_Region_Peru',
       'Country_Region_Russia', 'Country_Region_Spain',
       'Country_Region_Sweden', 'Country_Region_US', 'Country_Region_Ukraine',
       'Country_Region_United Kingdom', 'Lat', 'Long_', 'Confirmed', 'Deaths',
       'Incident_Rate', 'Case_Fatality_Ratio', 'Province_State_encoded',
       'Last_Update_encoded', 'date_encoded'],
      dtype='ob

In [ ]:
%%time
from sklearn.model_selection import train_test_split


def getDataTrain(data, target):

  df_train_encoded_copy =  data.copy()
  Ytrain =  df_train_encoded_copy[target]
  df_train_encoded_copy = df_train_encoded_copy.drop([target], axis=1 )


  ss = StandardScaler()
  ss = ss.fit(df_train_encoded_copy)

  Xtrain0 = ss.transform(df_train_encoded_copy)

  Xtrain = np.nan_to_num(Xtrain0)


  ##测试集
  ss1 = StandardScaler()
  ss1=ss1.fit(Ytrain.values.reshape(-1,1))
  Ytrain=ss1.transform(Ytrain.values.reshape(-1,1))

  return Xtrain, Ytrain, ss1

def getDataTrainEval( data,target, eval_size):
  train, eval = train_test_split(data, test_size=eval_size)
  Xtrain, Ytrain, ssTrain = getDataTrain(train, target)
  Xeval, Yeval, ssEval = getDataTrain(eval, target)
  return Xtrain, Ytrain, ssTrain, Xeval, Yeval , ssEval

def getDataTest(target):

  df_test_encoded_missing_copy =  df_test_encoded_missing.copy()

  Y_DS =   df_test_encoded[target]
  df_test_encoded_missing_copy = df_test_encoded_missing_copy.drop([target], axis=1 )

  ss = StandardScaler()
  ss = ss.fit(df_test_encoded_missing_copy)

  X_DS0 = ss.transform(df_test_encoded_missing_copy)

  X_DS = np.nan_to_num(X_DS0)

  ##测试集
  ss3 = StandardScaler()
  ss3 = ss3.fit(Y_DS.values.reshape(-1,1))
  Y_DS=ss3.transform(Y_DS.values.reshape(-1,1))
  return X_DS, Y_DS, ss3

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 11.9 µs


## 6. Train / test  Model

In [ ]:
%%time

i = 0
eval_size = 0.1
columnas = df_train_encoded.columns[i:]
try :
  filled_data = pd.read_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_XGBoost_Desarrollo='+str(MODO_DESARROLLO)+'.csv')
except:
  filled_data = pd.DataFrame()
  pass


for column in  columnas:
  print("Column: ", column)
  Xtrain, Ytrain, ss1, Xeval, Yeval , ssEval = getDataTrainEval(df_train_encoded, column, eval_size)

  gradient_boosting_regressoer = GradientBoostingRegressor(n_estimators=70, max_features=0.24270626876974666,
                                                        max_depth=6, min_samples_split=12,
                                                        min_samples_leaf=8,
                                                        learning_rate=0.028834960711679167, random_state=909, verbose = 1)

  gradient_boosting_regressoer.fit(Xtrain, Ytrain)

  Yeval_predict = gradient_boosting_regressoer.predict(Xeval).reshape(-1,1)
  Yeval_predict = ssEval.inverse_transform(Yeval_predict)
  Yeval = ssEval.inverse_transform(Yeval)

  rmse_train = sqrt(metrics.mean_squared_error(Yeval, Yeval_predict))
  print('RMSE training: ', rmse_train)



  print("Termino entrenamiento i=", i)
  print("------------------------------------------------")

  X_DS, Y_DS, ss3 = getDataTest(column)


  Y_DS_predict = gradient_boosting_regressoer.predict(X_DS).reshape(-1,1)
  Y_DS_predict = ss3.inverse_transform(Y_DS_predict)
  Y_DS = ss3.inverse_transform(Y_DS)

  rmse_column =  sqrt(metrics.mean_squared_error(Y_DS, Y_DS_predict))
  print('RMSE column for ', (column),(rmse_column))

  output_list = [item[0] for item in Y_DS_predict]
  filled_data[column] = output_list
  filled_data.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_XGBoost_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

  i = i+1
  print("Termino test i=", i)
  print("------------------------------------------------")
print("Learning Finished!")



Column:  Country_Region_Australia


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           0.9529           50.64s
         2           0.9091            2.10m
         3           0.8576            2.28m
         4           0.8095            2.40m
         5           0.7675            2.67m
         6           0.7338            2.74m
         7           0.6921            2.93m
         8           0.6754            2.92m
         9           0.6370            2.88m
        10           0.6050            3.00m
        20           0.3666            2.48m
        30           0.2213            2.03m
        40           0.1295            1.56m
        50           0.0773            1.05m
        60           0.0483           31.64s
        70           0.0286            0.00s
RMSE training:  0.014575449509276675
Termino entrenamiento i= 0
------------------------------------------------
RMSE column for  Country_Region_Australia 0.026494048085475685
Termino test i= 1
----------------------------------------

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           0.9754            2.89m
         2           0.9199            1.97m
         3           0.8677            2.14m
         4           0.8183            2.31m
         5           0.7718            2.47m
         6           0.7342            2.61m
         7           0.6928            2.72m
         8           0.6684            2.67m
         9           0.6304            2.67m
        10           0.6066            2.77m
        20           0.3465            2.36m
        30           0.1949            1.99m
        40           0.1107            1.52m
        50           0.0623            1.04m
        60           0.0361           31.06s
        70           0.0202            0.00s
RMSE training:  0.044439725808610844
Termino entrenamiento i= 1
------------------------------------------------
RMSE column for  Country_Region_Belgium 0.03827164694914909
Termino test i= 2
-------------------------------------------

...

      Iter       Train Loss   Remaining Time 
         1           0.9977            2.93m
         2           0.9908            3.35m
         3           0.9827            3.03m
         4           0.9685            3.05m
         5           0.9666            2.99m
         6           0.9558            2.98m
         7           0.9470            2.92m
         8           0.9453            2.82m
         9           0.9343            2.74m
        10           0.9226            2.75m
        20           0.8479            2.38m
        30           0.7961            1.96m
        40           0.7353            1.49m
        50           0.6898           59.71s
        60           0.6439           30.15s
        70           0.5959            0.00s
RMSE training:  121.52075806363806
Termino entrenamiento i= 31
------------------------------------------------
RMSE column for  Province_State_encoded 135.09299758237634
Termino test i= 32
--------------------------------------------

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           0.9718            3.40m
         2           0.9320            3.44m
         3           0.9015            2.97m
         4           0.8663            3.03m
         5           0.8313            3.02m
         6           0.8231            2.91m
         7           0.7933            2.90m
         8           0.7683            2.83m
         9           0.7402            2.85m
        10           0.7149            2.86m
        20           0.5414            2.41m
        30           0.4146            2.00m
        40           0.3311            1.52m
        50           0.2825            1.02m
        60           0.2480           30.26s
        70           0.2228            0.00s
RMSE training:  115.08629489337017
Termino entrenamiento i= 32
------------------------------------------------
RMSE column for  Last_Update_encoded 155.33535807089308
Termino test i= 33
-----------------------------------------------

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           0.9801            2.54m
         2           0.9567            2.74m
         3           0.9400            2.92m
         4           0.9209            2.98m
         5           0.8779            2.86m
         6           0.8589            2.96m
         7           0.8514            2.90m
         8           0.8340            2.78m
         9           0.8075            2.82m
        10           0.7881            2.83m
        20           0.6757            2.37m
        30           0.5497            1.96m
        40           0.4524            1.48m
        50           0.3906           59.21s
        60           0.3406           29.48s
        70           0.2797            0.00s
RMSE training:  122.088682472563
Termino entrenamiento i= 33
------------------------------------------------
RMSE column for  date_encoded 156.73905043169142
Termino test i= 34
------------------------------------------------
Learnin

## Guardo el resultados

## Calculo RMSE

In [ ]:
filled_data = pd.read_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_XGBoost_Desarrollo='+str(MODO_DESARROLLO)+'.csv')
filled_data.reset_index(drop=True, inplace=True)
filled_data

,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,0.000248,0.000346,0.000946,0.000694,0.000504,0.001134,0.000920,0.000048,0.000359,0.000512,...,0.000701,38.141013,-87.410370,17497.165792,294.637969,24663.575984,1.450461,304.491091,491.997979,335.890687
1,0.000256,0.000345,0.000928,0.000469,0.000507,0.001133,0.000814,0.000048,0.000346,0.000512,...,0.000701,38.258637,-87.344021,19319.341174,386.990508,23351.861921,1.459924,390.040655,471.042374,330.445581
2,0.000240,0.000341,0.000930,0.000651,0.000702,0.001134,0.000677,0.000048,0.000374,0.000512,...,0.000701,37.292180,-86.168836,28758.196757,461.312761,23791.671185,1.289954,363.153107,537.332820,352.571779
3,0.000238,0.000345,0.000947,0.000507,0.000531,0.001133,0.008744,0.000048,0.000209,0.000321,...,0.000684,36.679671,-87.650656,14710.036362,347.865326,8614.368502,2.033401,346.794170,129.785625,171.620715
4,0.000233,0.000345,0.000933,0.000497,0.000495,0.001133,0.000918,0.000048,0.000384,0.000512,...,0.000700,38.704118,-87.333543,18042.263751,242.878938,23809.766728,1.387372,379.399631,303.474996,323.539492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897662,0.000252,0.000337,0.000952,0.000627,0.000482,0.001134,0.000105,0.000048,0.000425,0.000582,...,0.000702,38.226040,-85.187874,40137.661907,564.783551,11028.607072,1.711607,330.926489,217.062346,369.668171
897663,0.000251,0.000342,0.000958,0.000497,0.000637,0.001134,0.000750,0.000048,0.000441,0.000512,...,0.000705,38.217769,-91.579528,23650.264913,306.184004,11701.392958,1.587961,369.371834,445.115570,370.317877
897664,0.000251,0.000346,0.000931,0.001167,0.000505,0.001133,0.000707,0.000048,0.000327,0.000818,...,0.000704,38.258637,-95.337667,21094.590929,316.533086,22118.859773,1.264267,372.354568,523.560622,355.242853
897665,0.000240,0.000340,0.000921,0.000800,0.000537,0.001128,0.000606,0.000048,0.000468,0.000564,...,0.000680,37.203953,-84.799632,46375.365910,626.661338,12242.842327,1.614694,307.952136,429.935839,449.507400


In [ ]:
rmse = utils.rmse(df_test_encoded.to_numpy(), filled_data.to_numpy(), mask_test_open)
print("rmse :", rmse)

rmse : 15544.636352413794


In [ ]:
df_test_encoded, filled_data = utils.obtener_df_imputados(df_test_encoded, filled_data, mask_test_open)

## decodifico

In [ ]:
print("inicio de decoding")

imputed_data_decoded=utils.decode(filled_data, df_train_encoded, PATH_DATA_PROCESADA)

print("imputed_data_decoded.head(5)")
print(imputed_data_decoded.head(5))

print("imputed_data_decoded.to_csv")
imputed_data_decoded.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_XGBoost_full_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

print("fin de decoding")

inicio de decoding


897667it [01:56, 7725.04it/s]


imputed_data_decoded.head(5)
  Province_State Country_Region          Last_Update        Lat    Long_  \
0          Texas             US  2021-01-02 05:22:33  41.123513 -83.1278   
1          Texas             US  2021-01-02 05:22:33  38.258637 -75.2996   
2          Texas             US  2021-01-02 05:22:33  35.780773 -86.1688   
3          Texas             US  2021-01-02 05:22:33  33.618867 -87.6507   
4          Texas             US  2021-01-02 05:22:33  43.720997 -87.9458   

   Confirmed  Deaths  Incident_Rate  Case_Fatality_Ratio        date  
0      12750     230   23107.035413             1.788235  01-01-2021  
1      10041     167   19549.853001             1.663181  01-01-2021  
2      28758     406   30784.732824             1.325795  01-01-2021  
3        543      27    4167.013225             5.389222  01-01-2021  
4      15026     171   13027.570661             1.138027  01-01-2021  
imputed_data_decoded.to_csv
fin de decoding
